In [1]:
import geopandas as gpd
import json

cutblock_df = gpd.read_file("data/alberta_cutblocks.shp")
ab_df = gpd.read_file("data/alberta_boundary.shp")

In [2]:
fmu_name = "S10"
fmu = cutblock_df.query(f"FMU_NAME=='{fmu_name}'")
fmu = fmu.dissolve(by='FMU_NAME')
fmu.centroid

FMU_NAME
S10    POINT (479085.290 6251500.838)
dtype: geometry

In [3]:
from ipyleaflet import Map, GeoData, basemaps, ScaleControl, projections
from ipywidgets import Layout

centroid = fmu.centroid.to_crs(epsg=4326) # Get the centroid of the fmu in decimal degrees
center = centroid[0] # get the first point
center = (center.y, center.x) # Convert it to (lat, lon)

m = Map(center=center, zoom=9.5, 
        basemap=basemaps.Esri.WorldImagery, 
        layout=Layout(width="100%", height="600px"))

cutblocks = GeoData(geo_dataframe=fmu.to_crs(epsg=4326),
                    style={'color': '#00FF00', 'fillOpacity': 0, 'weight': 3},
                    name=f'Cutblocks (FMU = {fmu_name})')

m.add_layer(cutblocks)
m.add_control(ScaleControl(position='bottomleft'))
m

Map(center=[56.430113818355224, -115.33925880126449], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [4]:
import ee
ee.Initialize()
gfc = ee.Image("UMD/hansen/global_forest_change_2019_v1_7")

In [5]:
vis_params = {
    "bands": ["lossyear"],
    "min": 0,
    "max": 19,
    "palette": ["yellow", "red"]
}

url = gfc.getMapId(vis_params)["tile_fetcher"].url_format

In [6]:
from ipyleaflet import TileLayer
tile_layer = TileLayer(url=url)
m.add_layer(tile_layer)

In [61]:
from pprint import pprint

In [90]:
jsonData = cutblock_df.head(1).to_crs(epsg=4326).to_json()
dictData = eval(jsonData)
coords = dictData['features'][0]['geometry']['coordinates'][0] # Take the coordinates from just the first cutblock
features = ee.FeatureCollection([ee.Feature(ee.Geometry.Polygon(coords), dictData['features'][0]['properties'])])

In [91]:
maineCounties = ee.FeatureCollection('TIGER/2016/Counties').filter(ee.Filter.eq('STATEFP', '23'))
maineMeansFeatures = gfc.reduceRegions(**{
  "collection": features,
  "reducer": ee.Reducer.mode(),
  "scale": 30,
})
pprint(ee.Feature(maineCounties.first()).getInfo())

{'geometry': {'coordinates': [[[-69.35567769200567, 45.07345934494386],
                               [-69.35540835250748, 45.072516760505955],
                               [-69.35536347311837, 45.07238215327114],
                               [-69.3552287951465, 45.07202305117844],
                               [-69.35477997147858, 45.07040724569856],
                               [-69.3543311894197, 45.06879149751589],
                               [-69.35388235605362, 45.06722058408237],
                               [-69.35383744547215, 45.06717564823827],
                               [-69.35383749178813, 45.067130787757954],
                               [-69.3538373930896, 45.067041030883885],
                               [-69.35379257554604, 45.066906374653826],
                               [-69.3537477118766, 45.06681657893239],
                               [-69.35374770912864, 45.06672684072095],
                               [-69.35365788455232, 45.066502437

                               [-68.77134218814115, 44.67660127088166],
                               [-68.7674821467031, 44.675928012152106],
                               [-68.76739234396501, 44.67588317800342],
                               [-68.76631518498114, 44.67570361009919],
                               [-68.76685380871477, 44.67467132204606],
                               [-68.76757188878432, 44.67323504255441],
                               [-68.76882863229145, 44.67072160229052],
                               [-68.77004054451872, 44.66825297585553],
                               [-68.77125235611338, 44.665739528859326],
                               [-68.77183582322326, 44.66457257213435],
                               [-68.77188073311305, 44.664527710196744],
                               [-68.77188078148966, 44.66443789687252],
                               [-68.76595610364932, 44.66336069660001],
                               [-68.7646993441603, 44.66318119

                               [-68.18022919100848, 45.61802826448082],
                               [-68.18036385762593, 45.61802824200638],
                               [-68.1809024024895, 45.6179385137689],
                               [-68.18157564691366, 45.617803884733966],
                               [-68.18238360357527, 45.61771411820888],
                               [-68.18319149127315, 45.61757945176856],
                               [-68.18395451322803, 45.617444794181445],
                               [-68.18431362346107, 45.61739991735344],
                               [-68.18512148353159, 45.61726523416555],
                               [-68.1852561253238, 45.617220396505424],
                               [-68.18592937742596, 45.61713062862505],
                               [-68.18673724873773, 45.61695105874191],
                               [-68.1871412917014, 45.61690617317561],
                               [-68.1875900679486, 45.61681644116

                               [-68.82066887452439, 46.26264210672486],
                               [-68.82071376496633, 46.260442803429605],
                               [-68.82080357254745, 46.25142122636954],
                               [-68.82080351758833, 46.250074726344174],
                               [-68.82084842007137, 46.25002989105897],
                               [-68.82080350307308, 46.244688759457105],
                               [-68.82080346560325, 46.24266900824616],
                               [-68.82080349014036, 46.240649277149075],
                               [-68.82075858698298, 46.23867437179028],
                               [-68.82075863870475, 46.2366546599493],
                               [-68.82075869463249, 46.2346797640419],
                               [-68.82075859603994, 46.233826969297354],
                               [-68.8207586511305, 46.23274981118042],
                               [-68.82075863213127, 46.2307749

                               [-69.35262563913001, 45.07390817424333],
                               [-69.35267050875895, 45.07386328202768],
                               [-69.35280511907838, 45.07386330803636],
                               [-69.35285003738478, 45.07386330098411],
                               [-69.35293977824604, 45.0738632687839],
                               [-69.35302956550075, 45.073818369303886],
                               [-69.35307443462288, 45.0738184271988],
                               [-69.35316417517784, 45.07381839477274],
                               [-69.3532540113412, 45.07381838028564],
                               [-69.35482486171385, 45.07359401584681],
                               [-69.35567769200567, 45.07345934494386]]],
              'type': 'Polygon'},
 'id': '00000000000000000371',
 'properties': {'ALAND': 8798667306,
                'AWATER': 414015520,
                'CBSAFP': '12620',
                'CLASSFP': 'H1',
 

In [79]:
pprint(ee.Feature(features.first()).getInfo())

{'geometry': {'coordinates': [[[-114.10415936781362, 58.60773113052933],
                               [-114.10411540246182, 58.60781044354863],
                               [-114.10397213003043, 58.60785954344807],
                               [-114.10332182167254, 58.60786070404458],
                               [-114.10317144834049, 58.6078892435864],
                               [-114.10292280362808, 58.60799614091214],
                               [-114.10273939864773, 58.608150870836106],
                               [-114.10285039941098, 58.60827918948483],
                               [-114.10286990572513, 58.608342536603395],
                               [-114.10285074261687, 58.60840501020636],
                               [-114.1027980187008, 58.60843077996839],
                               [-114.10270996762718, 58.60863774275288],
                               [-114.10277123962646, 58.608797277410225],
                               [-114.1029422117830

                               [-114.14935816045914, 58.59105900063533],
                               [-114.14924587968076, 58.5907903244407],
                               [-114.14925191968811, 58.590515448494855],
                               [-114.14893637929802, 58.58993632106202],
                               [-114.14893833377236, 58.58993336524581],
                               [-114.14905862056403, 58.58975146176478],
                               [-114.14928783680504, 58.58947788096408],
                               [-114.149401162333, 58.58910053259556],
                               [-114.14939693743636, 58.588695019286206],
                               [-114.14921419574462, 58.58804793681929],
                               [-114.14932118640557, 58.58795920652246],
                               [-114.14950497600867, 58.58796026505603],
                               [-114.14990801324925, 58.58814127732547],
                               [-114.15005045951176,

                               [-114.06310799191334, 58.590876758898844],
                               [-114.06297787404353, 58.590827817772066],
                               [-114.06288765869195, 58.590758508049646],
                               [-114.06257526483515, 58.59064653588489],
                               [-114.06219572197834, 58.59059597919435],
                               [-114.06148875247538, 58.59038156282639],
                               [-114.06135583492623, 58.590324598976515],
                               [-114.06119843213443, 58.59030302412653],
                               [-114.0610303998934, 58.590254244452645],
                               [-114.06095611624883, 58.5902531969721],
                               [-114.06055514983842, 58.59024754086295],
                               [-114.06031569108363, 58.59015775809778],
                               [-114.06005699520018, 58.58999801297403],
                               [-114.05924272102